# Tableaux semánticos

## Reglas para construcción de los tableaux

In [1]:
from utils import MainConective, Node

def alpha_rules(exp):
    con = MainConective(exp)
    if con[0] in '^':
        l, r = con[1], con[2]
        return [l, r]
    elif con[0] == '¬':
        con1 = MainConective(con[1])
        if con1[0] == 'v':
            return ['¬'+con1[1], '¬'+con1[2]]
        elif con1[0] == '->':
            return [con1[1], '¬'+con1[2]]
    else:
        pass
        
def beta_rules(exp):
    con = MainConective(exp)
    
    if con[0] == 'v':
        return [con[1], con[2]]
    elif con[0] == '->':
        return ['¬'+con[1], con[2]]
    elif con[0] == '¬':
        con1 = MainConective(con[1])
        if con1[0] == '^':
            return ['¬'+con1[1], '¬'+con1[2]]
    else:
        pass

def sigma_rules(exp):
    if '¬¬' == exp[:2]:
        return exp[2]
    elif '¬(¬' == exp[:3]:
        return exp[3]

In [2]:
print(alpha_rules('p^q'))
print(alpha_rules('¬(pvq)'))
print(alpha_rules('¬(p->q)'))

print(beta_rules('pvq'))
print(beta_rules('p->q'))
print(beta_rules('¬(p^q)'))

print(sigma_rules('¬(¬p)'))
print(sigma_rules('¬¬p'))

['p', 'q']
['¬p', '¬q']
['p', '¬q']
['p', 'q']
['¬p', 'q']
['¬p', '¬q']
p
p


## Construcción del árbol

In [3]:
class Node(object):
    def __init__(self, expression, leaf=False, closed=False, parent=None, tipo=None):
        self.exp = expression
        self.leaf = leaf
        self.closed = closed
        self.parent = parent
        self.tipo = tipo
    def __str__(self):
        if self.exp != None:
            return self.exp
        else:
            return ''
        
def check_leaf(expression):
    if alpha_rules(expression) == None and beta_rules(expression) == None:
        return True
    else:
        return False

In [4]:
def construct_tree(axioms):
    tree = [Node('root', tipo='alpha')]
    leafs = []
    for i,formula in enumerate(axioms):
        node = Node(formula, parent=tree[i], tipo='alpha')
        tree.append(node)

    for n in tree:
        alpha = alpha_rules(n.exp)
        if alpha != None:
            alpha_node1 = Node(alpha[0], parent=tree[-1], tipo='alpha')
            alpha_node2 = Node(alpha[1], parent=alpha_node1, tipo='alpha')
            tree.append(alpha_node1)
            tree.append(alpha_node2)
        else:
            pass
        
    for n in tree:
        beta = beta_rules(n.exp)
        if beta != None:
            if n.tipo == 'alpha':
                beta_node1 = Node(beta[0], parent=tree[-1], tipo='beta')
                beta_node2 = Node(beta[1], parent=tree[-1], tipo='beta')
            else:
                beta_node1 = Node(beta[0], parent=n, tipo='beta')
                beta_node2 = Node(beta[1], parent=n, tipo='beta')
            tree.append(beta_node1)
            tree.append(beta_node2)
        else:
            pass
        
    for i,n in enumerate(tree):
        if i < len(tree)-1:
            if sum([t.parent.exp == n.exp for t in tree[i+1:]]) == 0:
                n.leaf = True
        else:
            n.leaf = True

    return tree

In [5]:
tree = construct_tree(['(p^q)->(rvs)', 'q', '¬(rvs)', '¬¬p'])
    
for n in tree:
    print(n.parent, '\n\t-->', n) 

None 
	--> root
root 
	--> (p^q)->(rvs)
(p^q)->(rvs) 
	--> q
q 
	--> ¬(rvs)
¬(rvs) 
	--> ¬¬p
¬¬p 
	--> ¬r
¬r 
	--> ¬s
¬s 
	--> ¬p^q
¬s 
	--> rvs
¬p^q 
	--> ¬p
¬p^q 
	--> ¬q
rvs 
	--> r
rvs 
	--> s


## Revisión de las hojas

In [6]:
def check_closed_leafs(tree):
    checked = []
    for node in tree[::-1]:
        if node.leaf == True:
            expression = node.exp
            comparisson_node = node
            while comparisson_node.parent.exp != 'root':
                if expression == '¬'+comparisson_node.exp or '¬'+expression == comparisson_node.exp:
                    node.closed = True
                else:
                    pass
                comparisson_node = comparisson_node.parent
            checked.append(node.closed)

            print('Nodo hoja: {}; ¿cerrado?: {}'.format(node, node.closed))
    
    return all(checked)
    
check_closed_leafs(tree)

Nodo hoja: s; ¿cerrado?: True
Nodo hoja: r; ¿cerrado?: True
Nodo hoja: ¬q; ¿cerrado?: True
Nodo hoja: ¬p; ¿cerrado?: True


True